#**Dataset**

##Downloading the dataset

In [43]:
import os

if not os.path.isfile("kaggle.json"):
    raise FileNotFoundError("The 'kaggle.json' file is not found in the current directory. Upload the file to download the dataset")

In [44]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle': No such file or directory


In [45]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

  0% 0.00/199k [00:00<?, ?B/s]
100% 199k/199k [00:00<00:00, 77.3MB/s]


In [46]:
!unzip house-prices-advanced-regression-techniques.zip

Archive:  house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


##Reading the data

In [47]:
import pandas as pd
import numpy as np

In [48]:
df = pd.read_csv("/content/train.csv")

In [49]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [51]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


##Data Preprocessing

###Checking for NaN values

In [ ]:
df['LotFrontage'].isna

<bound method Series.isna of 0       65.0
1       80.0
2       68.0
3       60.0
4       84.0
        ... 
1455    62.0
1456    85.0
1457    66.0
1458    68.0
1459    75.0
Name: LotFrontage, Length: 1460, dtype: float64>

In [52]:
missing_data = pd.DataFrame({'total_missing': df.isnull().sum(),'data_type': df.dtypes})
missing_data

,total_missing,data_type
Id,0,int64
MSSubClass,0,int64
MSZoning,0,object
LotFrontage,259,float64
LotArea,0,int64
...,...,...
MoSold,0,int64
YrSold,0,int64
SaleType,0,object
SaleCondition,0,object


In [53]:
missing_data.iloc[50:,:]

,total_missing,data_type
HalfBath,0,int64
BedroomAbvGr,0,int64
KitchenAbvGr,0,int64
KitchenQual,0,object
TotRmsAbvGrd,0,int64
Functional,0,object
Fireplaces,0,int64
FireplaceQu,690,object
GarageType,81,object
GarageYrBlt,81,float64


In [ ]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [ ]:
df['Alley'].unique()

array([nan, 'Grvl', 'Pave'], dtype=object)

In [ ]:
null_indices = df[df['MasVnrType'].isnull()].index #Masonry veneer type
null_indices

Int64Index([234, 529, 650, 936, 973, 977, 1243, 1278], dtype='int64')

In [ ]:
df.loc[null_indices,'MasVnrArea'] # as Masonry veneer type is Null, its area is also null

234    NaN
529    NaN
650    NaN
936    NaN
973    NaN
977    NaN
1243   NaN
1278   NaN
Name: MasVnrArea, dtype: float64

In [10]:
df[df['BsmtExposure'].isnull()].index

Int64Index([  17,   39,   90,  102,  156,  182,  259,  342,  362,  371,  392,
             520,  532,  533,  553,  646,  705,  736,  749,  778,  868,  894,
             897,  948,  984, 1000, 1011, 1035, 1045, 1048, 1049, 1090, 1179,
            1216, 1218, 1232, 1321, 1412],
           dtype='int64')

In [26]:
not_common = df[df['BsmtFinType2'].isnull()].index.difference(df[df['BsmtExposure'].isnull()].index)

# Print the result
print(not_common)

Int64Index([332], dtype='int64')


In [27]:
df.loc[not_common,["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinSF1","BsmtFinType2"]]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2
332,Gd,TA,No,GLQ,1124,NaN


In [44]:
df['Electrical'].unique()

array(['SBrkr', 'FuseF', 'FuseA', 'FuseP', 'Mix', nan], dtype=object)

In [16]:
df.loc[df[df['BsmtExposure'].isnull()].index,["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinSF1","BsmtFinType2"]].style.highlight_null("red")
# BsmtQual	37	object
# BsmtCond	37	object
# BsmtExposure	38	object - no basement
# BsmtFinType1	37	object
# BsmtFinSF1	0	int64
# BsmtFinType2	38

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2
17,nan,nan,nan,nan,0,nan
39,nan,nan,nan,nan,0,nan
90,nan,nan,nan,nan,0,nan
102,nan,nan,nan,nan,0,nan
156,nan,nan,nan,nan,0,nan
182,nan,nan,nan,nan,0,nan
259,nan,nan,nan,nan,0,nan
342,nan,nan,nan,nan,0,nan
362,nan,nan,nan,nan,0,nan
371,nan,nan,nan,nan,0,nan


In [18]:
len(df.loc[df[df['BsmtExposure'].isnull()].index,["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinSF1","BsmtFinType2"]])

38

In [31]:
for col in ["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2"]:
  print(col+" : "+str(df[col].unique()))


# df[["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinSF1","BsmtFinType2"]]

BsmtQual : ['Gd' 'TA' 'Ex' nan 'Fa']
BsmtCond : ['TA' 'Gd' nan 'Fa' 'Po']
BsmtExposure : ['No' 'Gd' 'Mn' 'Av' nan]
BsmtFinType1 : ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
BsmtFinType2 : ['Unf' 'BLQ' nan 'ALQ' 'Rec' 'LwQ' 'GLQ']


In [55]:
df.loc[df[df['FireplaceQu'].isnull()].index,["Fireplaces","FireplaceQu"]].style.highlight_null("red")

,Fireplaces,FireplaceQu
0,0,nan
5,0,nan
10,0,nan
12,0,nan
15,0,nan
17,0,nan
18,0,nan
19,0,nan
26,0,nan
29,0,nan


In [74]:
display(df.loc[df[df["GarageType"].isnull()].index,['GarageType','GarageYrBlt','GarageFinish','GarageCars','GarageCars','GarageQual','GarageCond']])

,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageCars,GarageQual,GarageCond
39,NaN,NaN,NaN,0,0,NaN,NaN
48,NaN,NaN,NaN,0,0,NaN,NaN
78,NaN,NaN,NaN,0,0,NaN,NaN
88,NaN,NaN,NaN,0,0,NaN,NaN
89,NaN,NaN,NaN,0,0,NaN,NaN
...,...,...,...,...,...,...,...
1349,NaN,NaN,NaN,0,0,NaN,NaN
1407,NaN,NaN,NaN,0,0,NaN,NaN
1449,NaN,NaN,NaN,0,0,NaN,NaN
1450,NaN,NaN,NaN,0,0,NaN,NaN


### Common function to prepare the dataset (Train & Test)

In [ ]:
def prepare_dataset(df:pd.DataFrame)-> pd.DataFrame:
  """
    Function to prepare the dataset for training and testing.
    Performs label encoding.
  """

  #mapping  options to numbers
  mapping = {
      'Alley': {np.nan: 0, 'Grvl': 1, 'Pave': 2},
      'MasVnrType': {'BrkCmn': 4, 'BrkFace': 3, 'CBlock': 2, np.nan: 0, 'Stone': 1},
      'BsmtQual': {'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, np.nan: 0},
      'BsmtCond': {'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0, np.nan: 0},
      'BsmtExposure': {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, np.nan: 0},
      'BsmtFinType1': {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, np.nan: 0},
      'BsmtFinType2': {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, np.nan: 0}
  }

  df = df.replace(mapping)

  df['LotFrontage'].fillna(0, inplace=True) #na values of lot frontage will be replaced with zero
  df['MasVnrArea'].fillna(0, inplace=True) #na values of MasVnrArea will be replaced with zero as its Masonry veneer type is null

  df['Electrical'].fillna(0, inplace=True)
  df['FireplaceQu'].fillna(0, inplace=True)

  df['GarageType'].fillna(0, inplace=True)
  df['GarageFinish'].fillna(0, inplace=True)
  df['GarageQual'].fillna(0, inplace=True)
  df['GarageCond'].fillna(0, inplace=True)


  df.drop(columns=['GarageYrBlt'])



In [38]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Create a DataFrame with categorical variables
data = {
    'Color': ['Red', 'Blue', 'Green', 'Red', 'Blue'],
    'Size': ['Small', 'Large', 'Medium', 'Medium', 'Small'],
    'Shape': ['Circle', 'Square', 'Triangle', 'Square', 'Circle']
}

df = pd.DataFrame(data)

# Create an instance of the LabelEncoder
encoder = LabelEncoder()

# Apply label encoding to each column
df_encoded = df.apply(encoder.fit_transform)

# Print the encoded DataFrame
print(df_encoded)

   Color  Size  Shape
0      2     2      0
1      0     0      1
2      1     1      2
3      2     1      1
4      0     2      0


In [39]:
test = pd.DataFrame({
    'Color': ['Red'],
    'Size': ['Small'],
    'Shape': ['Circle']
})

In [40]:
test

,Color,Size,Shape
0,Red,Small,Circle


In [42]:
encoder.transform(test['Color'])

ValueError: ignored